2. RFM task
    1. Koristiti podatke o transakcijama računa (folder `data` fajl `trans.csv`) i povezati skup podataka sa podacima o korisnicima.
    2. Podeliti klijente (kolonu `client_id`) u 5 grupa i koristeći rfm_group_mapping funkciju (predavanja Segmentacija korisnika).
    3. Recency da se računa kao broj dana od 10.05.2019. godine.
    4. Frequency kao broj transakcija.
    5. Monetary kao prosečna vrednost transakcije.
    6. Prikazati prosečne vrednosti RFM kategorija po svakog grupi.

In [3]:
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

acc_cl= pd.read_csv('data/client_accounts.csv')
acc_cl.head(20)

,account_id,client_id
0,1,1
1,2,2
2,3,2
3,4,3
4,5,3
5,6,4
6,7,5
7,8,6
8,9,7
9,10,8


In [4]:
acc_cl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5369 entries, 0 to 5368
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   account_id  5369 non-null   int64
 1   client_id   5369 non-null   int64
dtypes: int64(2)
memory usage: 84.0 KB


In [5]:
trans= pd.read_csv('data/trans.csv')
trans.head(10)

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,695247,2378,2017-09-10 03:24:33,INCOME,DEPOSIT,700.0,700.0,NaN,NaN,NaN
1,171812,576,2017-09-14 03:24:33,INCOME,DEPOSIT,900.0,900.0,NaN,NaN,NaN
2,207264,704,2017-09-21 03:24:33,INCOME,DEPOSIT,1000.0,1000.0,NaN,NaN,NaN
3,1117247,3818,2017-09-06 03:24:33,INCOME,DEPOSIT,600.0,600.0,NaN,NaN,NaN
4,579373,1972,2017-09-19 03:41:13,INCOME,DEPOSIT,400.0,400.0,NaN,NaN,NaN
5,771035,2632,2017-09-12 03:41:13,INCOME,DEPOSIT,1100.0,1100.0,NaN,NaN,NaN
6,452728,1539,2017-09-08 03:57:53,INCOME,DEPOSIT,600.0,600.0,NaN,NaN,NaN
7,725751,2484,2017-09-07 03:57:53,INCOME,DEPOSIT,1100.0,1100.0,NaN,NaN,NaN
8,497211,1695,2017-09-22 03:57:53,INCOME,DEPOSIT,200.0,200.0,NaN,NaN,NaN
9,232960,793,2017-09-08 03:57:53,INCOME,DEPOSIT,800.0,800.0,NaN,NaN,NaN


In [6]:
trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1056320 entries, 0 to 1056319
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   trans_id    1056320 non-null  int64  
 1   account_id  1056320 non-null  int64  
 2   date        1056320 non-null  object 
 3   type        1056320 non-null  object 
 4   operation   873206 non-null   object 
 5   amount      1056320 non-null  float64
 6   balance     1056320 non-null  float64
 7   k_symbol    574439 non-null   object 
 8   bank        273508 non-null   object 
 9   account     295389 non-null   float64
dtypes: float64(3), int64(2), object(5)
memory usage: 80.6+ MB


In [7]:
trans['date']=pd.to_datetime(trans['date'])
trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1056320 entries, 0 to 1056319
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   trans_id    1056320 non-null  int64         
 1   account_id  1056320 non-null  int64         
 2   date        1056320 non-null  datetime64[ns]
 3   type        1056320 non-null  object        
 4   operation   873206 non-null   object        
 5   amount      1056320 non-null  float64       
 6   balance     1056320 non-null  float64       
 7   k_symbol    574439 non-null   object        
 8   bank        273508 non-null   object        
 9   account     295389 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(4)
memory usage: 80.6+ MB


In [8]:
df= pd.merge(acc_cl, trans, on='account_id', how='inner')
df.head()

,account_id,client_id,trans_id,date,type,operation,amount,balance,k_symbol,bank,account
0,1,1,1,2018-05-12 03:54:33,INCOME,DEPOSIT,1000.0,1000.0,NaN,NaN,NaN
1,1,1,5,2018-05-02 04:37:53,INCOME,TRANSFER FROM ACCOUNT,3679.0,4679.0,NaN,AB,41403269.0
2,1,1,199,2018-05-01 07:24:33,INCOME,DEPOSIT,12600.0,17279.0,NaN,NaN,NaN
3,1,1,3530438,2018-05-14 09:21:13,INCOME,NaN,19.2,17298.2,UROK,NaN,NaN
4,1,1,6,2018-05-15 08:24:33,INCOME,TRANSFER FROM ACCOUNT,3679.0,20977.2,NaN,AB,41403269.0


In [9]:
(df['amount']-(df['amount'].mean())).std()

9072.88111610982

In [10]:
df.groupby('client_id')['type'].nunique()
df.iloc[:,0:4]

,account_id,client_id,trans_id,date
0,1,1,1,2018-05-12 03:54:33
1,1,1,5,2018-05-02 04:37:53
2,1,1,199,2018-05-01 07:24:33
3,1,1,3530438,2018-05-14 09:21:13
4,1,1,6,2018-05-15 08:24:33
...,...,...,...,...
877277,11362,9227,3424092,2019-04-30 14:47:53
877278,11362,9227,3424044,2019-04-25 15:21:13
877279,11362,9227,3424236,2019-04-29 15:54:33
877280,11362,9227,3424248,2019-04-30 17:51:13


In [11]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 877282 entries, 0 to 877281
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   account_id  877282 non-null  int64         
 1   client_id   877282 non-null  int64         
 2   trans_id    877282 non-null  int64         
 3   date        877282 non-null  datetime64[ns]
 4   type        877282 non-null  object        
 5   operation   727053 non-null  object        
 6   amount      877282 non-null  float64       
 7   balance     877282 non-null  float64       
 8   k_symbol    481651 non-null  object        
 9   bank        232847 non-null  object        
 10  account     246221 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(3), object(4)
memory usage: 73.6+ MB


In [12]:
df['date']=pd.to_datetime(df['date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 877282 entries, 0 to 877281
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   account_id  877282 non-null  int64         
 1   client_id   877282 non-null  int64         
 2   trans_id    877282 non-null  int64         
 3   date        877282 non-null  datetime64[ns]
 4   type        877282 non-null  object        
 5   operation   727053 non-null  object        
 6   amount      877282 non-null  float64       
 7   balance     877282 non-null  float64       
 8   k_symbol    481651 non-null  object        
 9   bank        232847 non-null  object        
 10  account     246221 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(3), object(4)
memory usage: 73.6+ MB


In [13]:
df_rfm = df.groupby('client_id').agg({'date': 'max', 'amount': ['mean', 'count'] })
df_rfm

date       amount      
                          max         mean count
client_id                                       
1         2019-05-08 13:57:53  1569.767782   239
2         2019-05-09 16:44:33  5792.438824   595
3         2019-05-08 14:37:53  1917.973333   270
4         2019-05-06 15:37:53  2632.386585   246
5         2019-05-05 14:14:33  9169.537692   130
...                       ...          ...   ...
9031      2019-05-08 17:07:53  7147.950000   146
9156      2019-05-08 11:17:53  3450.256766   303
9199      2019-05-08 15:54:33  7465.088000    75
9225      2019-05-08 18:24:33  7799.157143   378
9227      2019-05-08 11:34:33  3866.309593   344

[3239 rows x 3 columns]

In [14]:
df_rfm.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3239 entries, 1 to 9227
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   (date, max)      3239 non-null   datetime64[ns]
 1   (amount, mean)   3239 non-null   float64       
 2   (amount, count)  3239 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 101.2 KB


In [15]:
df_rfm.columns=['-'.join(x) for x in df_rfm.columns]
df_rfm

,date-max,amount-mean,amount-count
client_id,,,
1,2019-05-08 13:57:53,1569.767782,239
2,2019-05-09 16:44:33,5792.438824,595
3,2019-05-08 14:37:53,1917.973333,270
4,2019-05-06 15:37:53,2632.386585,246
5,2019-05-05 14:14:33,9169.537692,130
...,...,...,...
9031,2019-05-08 17:07:53,7147.950000,146
9156,2019-05-08 11:17:53,3450.256766,303
9199,2019-05-08 15:54:33,7465.088000,75


In [16]:
day = datetime.strptime('2019-05-10', '%Y-%m-%d')
day

datetime.datetime(2019, 5, 10, 0, 0)

In [17]:
df_rfm['days_from_date']=df_rfm['date-max'].apply(lambda x: (day-x).seconds/60)
df_rfm

,date-max,amount-mean,amount-count,days_from_date
client_id,,,,
1,2019-05-08 13:57:53,1569.767782,239,602.116667
2,2019-05-09 16:44:33,5792.438824,595,435.450000
3,2019-05-08 14:37:53,1917.973333,270,562.116667
4,2019-05-06 15:37:53,2632.386585,246,502.116667
5,2019-05-05 14:14:33,9169.537692,130,585.450000
...,...,...,...,...
9031,2019-05-08 17:07:53,7147.950000,146,412.116667
9156,2019-05-08 11:17:53,3450.256766,303,762.116667
9199,2019-05-08 15:54:33,7465.088000,75,485.450000


In [18]:
#mora sa jedan jer 1/veliak vr je blize nuli, odnosno transakcija je obavljena davno, a blize jedan skorijeb
df_rfm['days_from_date']=1/df_rfm['days_from_date']
df_rfm

,date-max,amount-mean,amount-count,days_from_date
client_id,,,,
1,2019-05-08 13:57:53,1569.767782,239,0.001661
2,2019-05-09 16:44:33,5792.438824,595,0.002296
3,2019-05-08 14:37:53,1917.973333,270,0.001779
4,2019-05-06 15:37:53,2632.386585,246,0.001992
5,2019-05-05 14:14:33,9169.537692,130,0.001708
...,...,...,...,...
9031,2019-05-08 17:07:53,7147.950000,146,0.002426
9156,2019-05-08 11:17:53,3450.256766,303,0.001312
9199,2019-05-08 15:54:33,7465.088000,75,0.002060


In [19]:
df_rfm['Recency']=pd.qcut(df_rfm['days_from_date'], 5, False)+1
df_rfm.head(30)

,date-max,amount-mean,amount-count,days_from_date,Recency
client_id,,,,,
1,2019-05-08 13:57:53,1569.767782,239,0.001661,2
2,2019-05-09 16:44:33,5792.438824,595,0.002296,3
3,2019-05-08 14:37:53,1917.973333,270,0.001779,2
4,2019-05-06 15:37:53,2632.386585,246,0.001992,3
5,2019-05-05 14:14:33,9169.537692,130,0.001708,2
6,2019-05-06 13:37:53,6743.724803,254,0.001607,2
7,2019-05-08 17:07:53,1929.180332,361,0.002426,4
8,2019-05-07 17:07:53,3797.383626,342,0.002426,4
9,2019-05-06 04:51:13,2044.823585,106,0.000870,1


In [20]:
df_rfm['Frequency']= pd.qcut(df_rfm['amount-count'],5,False)+1
df_rfm.head(20)

,date-max,amount-mean,amount-count,days_from_date,Recency,Frequency
client_id,,,,,,
1,2019-05-08 13:57:53,1569.767782,239,0.001661,2,3
2,2019-05-09 16:44:33,5792.438824,595,0.002296,3,5
3,2019-05-08 14:37:53,1917.973333,270,0.001779,2,3
4,2019-05-06 15:37:53,2632.386585,246,0.001992,3,3
5,2019-05-05 14:14:33,9169.537692,130,0.001708,2,2
6,2019-05-06 13:37:53,6743.724803,254,0.001607,2,3
7,2019-05-08 17:07:53,1929.180332,361,0.002426,4,4
8,2019-05-07 17:07:53,3797.383626,342,0.002426,4,4
9,2019-05-06 04:51:13,2044.823585,106,0.000870,1,1


In [21]:
df_rfm['Monetary']=pd.qcut(df_rfm['amount-mean'],5,False)+1
df_rfm.head(20)

,date-max,amount-mean,amount-count,days_from_date,Recency,Frequency,Monetary
client_id,,,,,,,
1,2019-05-08 13:57:53,1569.767782,239,0.001661,2,3,1
2,2019-05-09 16:44:33,5792.438824,595,0.002296,3,5,4
3,2019-05-08 14:37:53,1917.973333,270,0.001779,2,3,1
4,2019-05-06 15:37:53,2632.386585,246,0.001992,3,3,2
5,2019-05-05 14:14:33,9169.537692,130,0.001708,2,2,5
6,2019-05-06 13:37:53,6743.724803,254,0.001607,2,3,4
7,2019-05-08 17:07:53,1929.180332,361,0.002426,4,4,1
8,2019-05-07 17:07:53,3797.383626,342,0.002426,4,4,3
9,2019-05-06 04:51:13,2044.823585,106,0.000870,1,1,1


In [22]:
df_rfm['RFM'] = df_rfm[['Recency', 'Frequency', 'Monetary']].agg('prod', axis=1)
df_rfm.tail()

,date-max,amount-mean,amount-count,days_from_date,Recency,Frequency,Monetary,RFM
client_id,,,,,,,,
9031,2019-05-08 17:07:53,7147.950000,146,0.002426,4,2,4,32
9156,2019-05-08 11:17:53,3450.256766,303,0.001312,1,4,3,12
9199,2019-05-08 15:54:33,7465.088000,75,0.002060,3,1,4,12
9225,2019-05-08 18:24:33,7799.157143,378,0.002981,4,4,4,64
9227,2019-05-08 11:34:33,3866.309593,344,0.001341,1,4,3,12


In [23]:
rfm_vr= df_rfm['RFM'].value_counts()
rfm_vr=rfm_vr.reset_index()
rfm_vr.sort_values(by='RFM')

,RFM,count
26,1,43
12,2,117
17,3,78
1,4,188
21,5,65
9,6,153
4,8,175
25,9,57
10,10,146
0,12,233


In [24]:
df_rfm['RFM Cluster'] = pd.cut(df_rfm['RFM'], bins=[0, 8, 20, 50, 80, 125], labels=False) + 1
df_rfm

,date-max,amount-mean,amount-count,days_from_date,Recency,Frequency,Monetary,RFM,RFM Cluster
client_id,,,,,,,,,
1,2019-05-08 13:57:53,1569.767782,239,0.001661,2,3,1,6,1
2,2019-05-09 16:44:33,5792.438824,595,0.002296,3,5,4,60,4
3,2019-05-08 14:37:53,1917.973333,270,0.001779,2,3,1,6,1
4,2019-05-06 15:37:53,2632.386585,246,0.001992,3,3,2,18,2
5,2019-05-05 14:14:33,9169.537692,130,0.001708,2,2,5,20,2
...,...,...,...,...,...,...,...,...,...
9031,2019-05-08 17:07:53,7147.950000,146,0.002426,4,2,4,32,3
9156,2019-05-08 11:17:53,3450.256766,303,0.001312,1,4,3,12,2
9199,2019-05-08 15:54:33,7465.088000,75,0.002060,3,1,4,12,2


In [25]:
df_rfm.groupby('RFM Cluster')['RFM'].count()

RFM Cluster
1    819
2    959
3    951
4    395
5    115
Name: RFM, dtype: int64

In [26]:
df_rfm.groupby('RFM Cluster')[['Recency', 'Frequency', 'Monetary']].agg(['mean', 'max', 'min'])

Recency         Frequency          Monetary        
                 mean max min      mean max min      mean max min
RFM Cluster                                                      
1            1.896215   5   1  1.825397   5   1  1.885226   5   1
2            2.810219   5   1  2.753910   5   1  2.794578   5   1
3            3.386961   5   1  3.492114   5   1  3.497371   5   1
4            4.096203   5   3  4.291139   5   3  4.116456   5   3
5            4.756522   5   4  4.730435   5   4  4.704348   5   4

In [27]:
def rfm_group_mapping(R, F, M):
    if (R <= 2) & (F <= 2) & (M <= 2):
        return 'Lost'
    elif (1 <= R <= 3) & (1 <= F <= 3) & (1 <= M <= 5):
        return 'At Risk'
    elif (1 <= R <= 3) & (2 <= F <= 5) & (2 <= M <= 4):
        return 'Need Attention'
    elif (4 <= R <= 5) & (F <= 2) & (M <= 4):
        return 'New Customers'
    elif (4 <= R <= 5) & (4 <= F <= 5) & (4 <= M <= 5):
        return 'Champion'
    else: 
        return 'Other'

In [28]:
df_rfm['RFM Group'] = df_rfm.apply(lambda x: rfm_group_mapping(x['Recency'], x['Frequency'], x['Monetary']), axis=1)

In [29]:
df_rfm.head()

,date-max,amount-mean,amount-count,days_from_date,Recency,Frequency,Monetary,RFM,RFM Cluster,RFM Group
client_id,,,,,,,,,,
1,2019-05-08 13:57:53,1569.767782,239,0.001661,2,3,1,6,1,At Risk
2,2019-05-09 16:44:33,5792.438824,595,0.002296,3,5,4,60,4,Need Attention
3,2019-05-08 14:37:53,1917.973333,270,0.001779,2,3,1,6,1,At Risk
4,2019-05-06 15:37:53,2632.386585,246,0.001992,3,3,2,18,2,At Risk
5,2019-05-05 14:14:33,9169.537692,130,0.001708,2,2,5,20,2,At Risk


In [30]:
df_rfm['RFM Group'].value_counts()

RFM Group
Other             921
At Risk           917
Need Attention    507
New Customers     376
Lost              281
Champion          237
Name: count, dtype: int64

In [31]:
df_rfm.groupby('RFM Group')[['amount-mean', 'amount-count', 'days_from_date']].agg(['mean', 'max', 'min'])

amount-mean                            amount-count        \
                       mean           max          min         mean   max   
RFM Group                                                                   
At Risk         6267.585708  20842.650769   958.511111   178.719738   298   
Champion        9098.867735  19181.433939  5744.719484   438.303797   968   
Lost            2070.668056   3233.688068   898.971831   113.583630   199   
Need Attention  4802.750690   8563.011908  2114.215646   441.850099  1008   
New Customers   3604.414071   8298.960185   920.614650   126.603723   199   
Other           5909.227217  21062.613287   856.444321   332.225841   944   

                    days_from_date                      
                min           mean       max       min  
RFM Group                                               
At Risk          11       0.001675  0.002296  0.000727  
Champion        299       0.003860  0.005925  0.002332  
Lost             12       0.001410  0.001868  0.000725  
Need Attention  299       0.001730  0.002296  0.000754  
New Customers    25       0.003943  0.028209  0.002332  
Other            47       0.003368  0.005925  0.000736